# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

2024-01-03 01:45:53.908908: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 01:45:55.024264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 01:45:55.024354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 01:45:55.515271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 01:45:55.813377: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 01:45:55.816084: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Importing Data
Importing the data, and then splitting it into features and labels

In [2]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
features = data.drop(['H2avg', 'dex', 'O18Avg', 'Station', 'Long'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
features['Date'] = features['Date'].map(datetime.datetime.toordinal)

numFeatures = features.shape[1]

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [3]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

# Convert data into tensorflow datasets
trainDataset = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))
testDataset = tf.data.Dataset.from_tensor_slices((xTest, yTest))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [4]:
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [5]:
model.fit(
    trainDataset.batch(32),
    batch_size=32,
    epochs=1000,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

Epoch 1/1000


45/45 [==============================] - 3s 6ms/step - loss: 352.5029 - root_mean_squared_error: 18.7751 - mean_absolute_error: 16.5082
Epoch 2/1000
45/45 [==============================] - 0s 6ms/step - loss: 184.0821 - root_mean_squared_error: 13.5677 - mean_absolute_error: 10.3667
Epoch 3/1000
45/45 [==============================] - 0s 6ms/step - loss: 82.6990 - root_mean_squared_error: 9.0939 - mean_absolute_error: 7.2654
Epoch 4/1000
45/45 [==============================] - 0s 5ms/step - loss: 47.5366 - root_mean_squared_error: 6.8947 - mean_absolute_error: 5.4849
Epoch 5/1000
45/45 [==============================] - 0s 5ms/step - loss: 37.0109 - root_mean_squared_error: 6.0837 - mean_absolute_error: 4.8107
Epoch 6/1000
45/45 [==============================] - 0s 5ms/step - loss: 30.3485 - root_mean_squared_error: 5.5089 - mean_absolute_error: 4.3612
Epoch 7/1000
45/45 [==============================] - 0s 5ms/step - loss: 26.2996 - root_mean_squared_error: 5.1283 - mean_absolute

In [11]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features.loc[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

12/12 [==============================] - 0s 3ms/step


/tmp/ipykernel_615/3509207634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_615/3509207634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


## Model Export

In [12]:
# Save the model
model.save('printModel.keras')

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
